In [1]:
#Federated Learning

#Using PySyft
import torch as th
import syft as sy

#Creating a hook
hook = sy.TorchHook(th)

W0705 19:05:16.143622 140541110052672 secure_random.py:22] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow (1.14.1-dev20190517). Fix this by compiling custom ops.
W0705 19:05:16.169894 140541110052672 deprecation_wrapper.py:119] From /home/ayush/anaconda3/lib/python3.7/site-packages/tf_encrypted/session.py:28: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [2]:
#Creating a remote worker
bob = sy.VirtualWorker(hook, id='bob')

In [3]:
#To check objects associated with a worker
bob._objects

{}

In [4]:
#Sending data to a worker
x = th.tensor([1, 2, 3, 4, 5])
x = x.send(bob)

In [5]:
bob._objects

{21467232781: tensor([1, 2, 3, 4, 5])}

In [6]:
#Returns a pointer to remote object 
x

(Wrapper)>[PointerTensor | me:69682024423 -> bob:21467232781]

In [7]:
#Check the location where pointer is pointing
x.location

<VirtualWorker id:bob #tensors:1>

In [8]:
#Check the id of the worker
x.id_at_location

21467232781

In [9]:
#Check the id of the 
x.id

69682024423

In [10]:
#Check the owner info(in this case we are the owner)
x.owner

<VirtualWorker id:me #tensors:0>

In [11]:
#A local worker is created when PySyft is hooked to torch which sends instructions to other workers
hook.local_worker

<VirtualWorker id:me #tensors:0>

In [12]:
#Get information back from worker
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [13]:
bob._objects

{}

In [14]:
#Creating another virtual worker
alice = sy.VirtualWorker(hook, id='alice')

In [15]:
x = th.tensor([1, 2, 3, 4, 5])

In [16]:
bob._objects

{}

In [17]:
alice._objects

{}

In [18]:
x_ptr = x.send(bob, alice)

In [19]:
x_ptr

(Wrapper)>[MultiPointerTensor]
	-> (Wrapper)>[PointerTensor | me:98783168852 -> bob:18770660149]
	-> (Wrapper)>[PointerTensor | me:82098489514 -> alice:95355898858]

In [20]:
bob._objects

{18770660149: tensor([1, 2, 3, 4, 5])}

In [21]:
alice._objects

{95355898858: tensor([1, 2, 3, 4, 5])}

In [22]:
x_ptr.get()

[tensor([1, 2, 3, 4, 5]), tensor([1, 2, 3, 4, 5])]

In [23]:
x = th.tensor([1, 2, 3, 4, 5]).send(bob, alice)

In [24]:
x.get(sum_results=True)

tensor([ 2,  4,  6,  8, 10])

In [25]:
#Remote Arithematic
x = th.tensor([1, 2, 3, 4, 5]).send(bob)
y = th.tensor([1, 1, 1, 1, 1]).send(bob)

In [26]:
#Adding two tensors
z = x + y
z

(Wrapper)>[PointerTensor | me:50819945939 -> bob:62727783671]

In [27]:
z.get()

tensor([2, 3, 4, 5, 6])

In [29]:
#Adding two tensors
z = th.add(x, y)
z

(Wrapper)>[PointerTensor | me:69468359141 -> bob:27253582150]

In [30]:
z.get()

tensor([2, 3, 4, 5, 6])

In [37]:
#Creating tensors with gradients on
x = th.tensor([1., 2, 3, 4, 5], requires_grad = True).send(bob)
y = th.tensor([1., 2, 3, 4, 5], requires_grad = True).send(bob)

In [38]:
#Calculating the sum of the tensors and the final sum of resulting tensor
z = (x + y).sum()
z

(Wrapper)>[PointerTensor | me:23429338461 -> bob:69022466170]

In [39]:
z.backward()

(Wrapper)>[PointerTensor | me:64148806116 -> bob:4357305287]

In [40]:
x = x.get()
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [41]:
x.grad

tensor([1., 1., 1., 1., 1.])

In [60]:
#Creating a Simple Linear Model
inputs = th.tensor([[1., 1], [0, 1], [1, 0], [0, 0]], requires_grad=True).send(bob)
targets = th.tensor([[1.], [1], [0], [0]], requires_grad=True).send(bob)

In [61]:
weights = th.tensor([[0.], [0]], requires_grad=True).send(bob)

In [62]:
#Creating a training loop
epochs = 10
for e in range(epochs):
    preds = inputs.mm(weights)
    loss = ((preds - targets)**2).sum()
    loss.backward()
    
    weights.data.sub_(weights.grad * 0.1)  #Subtract gradient times learning rate from weights to update weights
    weights.grad *= 0
    
    print(loss.get().data)

tensor(2.)
tensor(0.5600)
tensor(0.2432)
tensor(0.1372)
tensor(0.0849)
tensor(0.0538)
tensor(0.0344)
tensor(0.0220)
tensor(0.0141)
tensor(0.0090)


In [97]:
#Garbage collection
bob.clear_objects()

<VirtualWorker id:bob #tensors:0>

In [98]:
bob._objects

{}

In [99]:
x = th.tensor([1, 2, 3, 4, 5]).send(bob)

In [100]:
bob._objects

{70295980117: tensor([1, 2, 3, 4, 5])}

In [101]:
del x  #Deleting the pointer to the tensor deletes the tensor

In [102]:
bob._objects

{}

In [103]:
x = th.tensor([1, 2, 3, 4, 5]).send(bob)
x

(Wrapper)>[PointerTensor | me:88546166281 -> bob:38055591167]

In [104]:
x.child.garbage_collect_data  
#If this is set to off then deleting the pointer to the tensor would not delete the tensor itself

True

In [105]:
bob._objects

{38055591167: tensor([1, 2, 3, 4, 5])}

In [106]:
x = "abcd"
x

'abcd'

In [107]:
bob._objects  #This is a feature of jupyter notebook that the tensor did not get deleted

{38055591167: tensor([1, 2, 3, 4, 5])}

In [108]:
del x

In [109]:
bob._objects  #Tensor is still not deleted

{38055591167: tensor([1, 2, 3, 4, 5])}